In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [19]:
n = 1000000

series = pd.Series(np.arange(n), index=pd.date_range("2023-01-01", periods=n, freq="s"))
series.iloc[int(n/2)] = np.nan

In [20]:
np.isnan(series).all()

np.False_

In [21]:
series.isna().to_numpy().all()

np.False_

In [22]:
%%timeit

np.isnan(series).all()

239 μs ± 3.69 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
%%timeit

series.isna().to_numpy().all()

214 μs ± 1.53 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
n = 1000000
n_cols = 1

df = pd.DataFrame(
    np.arange(n).reshape(-1, n_cols),
    index=pd.date_range("2023-01-01", periods=int(n/n_cols), freq="s"),
)
df

,0
2023-01-01 00:00:00,0
2023-01-01 00:00:01,1
2023-01-01 00:00:02,2
2023-01-01 00:00:03,3
2023-01-01 00:00:04,4
...,...
2023-01-12 13:46:35,999995
2023-01-12 13:46:36,999996
2023-01-12 13:46:37,999997
2023-01-12 13:46:38,999998


In [30]:
series = df.iloc[:, 0]
series

2023-01-01 00:00:00         0
2023-01-01 00:00:01         1
2023-01-01 00:00:02         2
2023-01-01 00:00:03         3
2023-01-01 00:00:04         4
                        ...  
2023-01-12 13:46:35    999995
2023-01-12 13:46:36    999996
2023-01-12 13:46:37    999997
2023-01-12 13:46:38    999998
2023-01-12 13:46:39    999999
Freq: s, Name: 0, Length: 1000000, dtype: int64

In [ ]:
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.exceptions import MissingValuesWarning
from skforecast.utils import check_preprocess_series
from skforecast.utils import check_preprocess_exog_multiseries
from skforecast.utils import align_series_and_exog_multiseries
from skforecast.utils import align_series_and_exog_multiseries_new
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series_2
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series_as_dict
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_as_dict
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_as_dict_datetime
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_predict

In [41]:
exog_dict = {
    '1': exog.copy(),
    '2': exog['exog_1'].copy()
}

input_series_is_dict = isinstance(series, dict)
series_dict, series_indexes = check_preprocess_series(series=series)

exog_dict, _ = check_preprocess_exog_multiseries(
                    input_series_is_dict = input_series_is_dict,
                    series_indexes       = series_indexes,
                    series_names_in_     = ['1', '2'],
                    exog                 = exog_dict,
                    exog_dict            = {'1': None, '2': None}
                )

In [45]:
n_series = 600
len_series = (2000, 5000)
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    #n = rng.integers(low=len_series[0], high=len_series[1])
    n=2000
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=n),
        index=pd.date_range(
            start='2010-01-01',
            periods=n,
            freq='h'
        ),
        name=f'series_{i}'
    )
series_dataframe = pd.DataFrame(series_dict, index=series_dict['series_0'].index)

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog


exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog   


print(f"Range of dates: "
    f"{np.min([series_dict[k].index.min() for k in series_dict.keys()])} - "
    f"{np.max([series_dict[k].index.max() for k in series_dict.keys()])}"
)

print(f"Range of dates for prediction: "
      f"{np.min([exog_dict_prediction[k].index.min() for k in exog_dict_prediction.keys()])} - "
    f"{np.max([exog_dict_prediction[k].index.max() for k in exog_dict_prediction.keys()])}"
)

Range of dates: 2010-01-01 00:00:00 - 2010-03-25 07:00:00
Range of dates for prediction: 2010-03-25 08:00:00 - 2010-03-29 11:00:00


In [48]:
input_series_is_dict = isinstance(series_dict, dict)
series_dict, series_indexes = check_preprocess_series(series=series_dict)
series_names_in_ = list(series_dict.keys())

exog_dict, _ = check_preprocess_exog_multiseries(
                    input_series_is_dict = input_series_is_dict,
                    series_indexes       = series_indexes,
                    series_names_in_     = series_names_in_,
                    exog                 = exog_dict,
                    exog_dict            = {serie: None for serie in series_names_in_}
                )

In [49]:
%%timeit

align_series_and_exog_multiseries(
    series_dict          = series_dict,
    input_series_is_dict = input_series_is_dict,
    exog_dict            = exog_dict
)

6.21 ms ± 81.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
%%timeit

align_series_and_exog_multiseries_new(
    series_dict          = series_dict,
    input_series_is_dict = input_series_is_dict,
    exog_dict            = exog_dict
)

6.34 ms ± 118 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
